<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/cadastro_retencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
#importar módulos
import pandas as pd

In [70]:
!pip install pyxlsb

In [71]:
#designar nomes de ficheiros e designar o mês
cadastro_nome="Cadastro_EADW.xlsb"
agentes_nome="AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx"
cav_sfid_nome="SFIDsvsCAVs_20240506124348.xlsb"

mes="202404"

In [72]:
#fazer upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()

Saving AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04.xlsx to AGENTES_TODOS_ENTIDADE_GESTORA_KC13219_29.04 (1).xlsx


Saving Cadastro_EADW.xlsb to Cadastro_EADW (1).xlsb


Saving SFIDsvsCAVs_20240506124348.xlsb to SFIDsvsCAVs_20240506124348.xlsb


In [73]:
#importar cadastro
cadastro=pd.read_excel(cadastro_nome, sheet_name=0)
cadastro2=cadastro.copy()
cadastro2.columns=cadastro.iloc[0, :]
cadastro2.columns

Index(['COD MÊS DE FECHO (YYYYMM)', 'COD UTILIZADOR', 'DSC UTILIZADOR',
       'COD RESPONSÁVEL', 'COD ENTIDADE', 'DSC FUNÇÃO', 'DSC CENTRO',
       'DSC PILOTO', 'COD SEGMENTO DE RETENÇÃO', 'QTY CARGA HORÁRIA POR DIA',
       'QTY HORAS TRABALHADAS NO MÊS',
       'QTY HORAS DE ABSENTISMO JUSTIFICADAS NO MÊS',
       'QTY HORAS DE ABSENTISMO INJUSTIFICADAS NO MÊS', 'CAV INBOUND',
       'CAV OUTBOUND'],
      dtype='object', name=0)

In [74]:
#importar ficheiro "agentes todos entidade gestora" do mês

agentes=pd.read_excel(agentes_nome, sheet_name=0)

In [75]:
agentes.columns

Index(['ID Agente', 'Agente', 'ID Entidade do Agente', 'Entidade do Agente',
       'ID Delegação do Agente', 'Delegação do Agente', 'ID Rede do Agente',
       'Rede do Agente', 'Canal de Venda do Agente',
       'Entidade Gestora (Agentes)', 'Segmento de Agente', 'Tipo do Agente',
       'Situação do Agente', 'NIF do Agente'],
      dtype='object')

In [76]:
#importar ficheiro sidsvscavs
sfids_cavs=pd.read_excel(cav_sfid_nome, sheet_name=0)

In [109]:
#formatar colunas na dataframe sfids_cavs

sfids_cavs.columns=[i.strip() for i in sfids_cavs.columns]

In [80]:
#filtrar dataframe para os dados do mês n

cadastro_mes=cadastro2[cadastro2.iloc[:, 0]==mes]


In [81]:
#criação colunas inbound e outbound - chaves representam colunas a criar, valores representam colunas no ficheiro entidade agente

def conversao (dici, str, df=cadastro_mes, df2=agentes):
  for i, j in dici.items():
    for cav in df["CAV " + str].unique().tolist():
      indice=df.loc[df["CAV " + str]==cav, :].index
      var=df2.loc[df2.iloc[:, 0]==cav, j].unique().tolist()[0]
      df.loc[indice, i]=var

  return df

#dicionario inb
dici_inb={"Delegação IN_EntidadeGestora":"ID Delegação do Agente", "Entidade (EG)": "ID Entidade do Agente",  "Tipo do Agente": "Tipo do Agente", "Rede (EG)- Inbound": "Rede do Agente",\
"Estado cav inbound": "Situação do Agente"}

conversao(dici_inb, "INBOUND")

#dicionario out

dici_out={"Delegação OUT_EntidadeGestora":"ID Delegação do Agente", "Rede (EG)- Outbound":"Rede do Agente", "Estado Cav Outbound": "Situação do Agente"}


conversao(dici_out, "OUTBOUND")


<ipython-input-81-a6016558c0fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indice, i]=var
<ipython-input-81-a6016558c0fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indice, i]=var
<ipython-input-81-a6016558c0fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,COD MÊS DE FECHO (YYYYMM),COD UTILIZADOR,DSC UTILIZADOR,COD RESPONSÁVEL,COD ENTIDADE,DSC FUNÇÃO,DSC CENTRO,DSC PILOTO,COD SEGMENTO DE RETENÇÃO,QTY CARGA HORÁRIA POR DIA,...,CAV INBOUND,CAV OUTBOUND,Delegação IN_EntidadeGestora,Entidade (EG),Tipo do Agente,Rede (EG)- Inbound,Estado cav inbound,Delegação OUT_EntidadeGestora,Rede (EG)- Outbound,Estado Cav Outbound
1,202404,KJEVANGELISTA,kelly jennifer teixeira evangelista,?,410052,Coordenador,Porto,Proactiva,-1,8,...,60090418,70090418,410052,410052.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,412030,Outbound II,ACTIVO
2,202404,AFELEITE,andré leite,?,410052,Coordenador,Porto,Proactiva,-1,8,...,60135317,70135317,410052,410052.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,410052,FIDELIZAÇÃO,ACTIVO
3,202404,U80025211,diogo martins oliveira duarte,?,201056,Coordenador,Lisboa,NaN,-1,8,...,80025211,70025211,201056,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,201056,FIDELIZAÇÃO,ACTIVO
4,202404,U80028220,ines alves ferreira lima teixeira,?,201056,Coordenador,Lisboa,NaN,-1,8,...,80028220,70028220,201056,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,201056,FIDELIZAÇÃO,ACTIVO
5,202404,U80010362,marco cesar moreira alves dias,?,203183,Coordenador,Porto,Inbound,-1,8,...,80010362,70010362,409303,203183.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,409303,FIDELIZAÇÃO,ACTIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,202404,U80037354,daniela alexandra ferreira da silva,U80028220,201056,Supervisor,Lisboa,Inbound,-1,8,...,80037354,70037354,409621,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,409621,FIDELIZAÇÃO,ACTIVO
384,202404,U80070712,ines raquel da luz cristovao,U80028220,201056,Supervisor,Lisboa,Blend Out,-1,8,...,80070712,60070712,411229,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,411229,FIDELIZAÇÃO,ACTIVO
385,202404,LABASILVA,lara barbosa silva,U80028220,201056,Supervisor,Lisboa,Inbound,-1,5,...,70141921,80141921,409624,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,409624,FIDELIZAÇÃO,ACTIVO
386,202404,MICRNASCIMEN,miranda cristina nascimento,U80028220,201056,Supervisor,Lisboa,Inbound,-1,8,...,80145713,70145713,411238,201056.0,Fidelização Inbound,FIDELIZAÇÃO,ACTIVO,411238,FIDELIZAÇÃO,ACTIVO


In [103]:
#criar colunas "CHAVE INB" e "CHAVE OUT" na dataframe cadastro_mes

for i, j, k in zip(["CAV INBOUND", "CAV OUTBOUND"], ["Delegação IN_EntidadeGestora", "Delegação OUT_EntidadeGestora"], ["COD ENTIDADE", "COD ENTIDADE"]):
  if i.find("OUTBOUND")!=-1:
    cadastro_mes["CHAVE OUT"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]
  else:
    cadastro_mes["CHAVE INB"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]




<ipython-input-103-7e7888b07dcc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cadastro_mes["CHAVE INB"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]
<ipython-input-103-7e7888b07dcc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cadastro_mes["CHAVE OUT"]=cadastro_mes[i] + cadastro_mes[j] + cadastro_mes[k]


In [118]:
#chave entidade, delegação, cav
sfids_cavs["Chave (del_ent_cav)"]=sfids_cavs["CAV Delegation"].astype("str") + sfids_cavs["CAV Entity"].astype("str") + sfids_cavs["CAV"].astype("str")

#chave cav, delegação, entidade

sfids_cavs["Chave (cav_ent_del)"]=sfids_cavs["CAV"].astype("str") + sfids_cavs["CAV Entity"].astype("str") + sfids_cavs["CAV Delegation"].astype("str")

In [119]:
sfids_cavs.columns

Index(['Dealer SFID', 'CAV', 'CAV Delegation', 'CAV Entity', 'CAV SFID Active',
       'Chave (del_ent_cav)', 'Chave (cav_ent_del)'],
      dtype='object')